In [1]:
#import keras
#print(keras.__version__)
import tensorflow as tf
print(tf.__version__)

2.7.0


In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input, GlobalAveragePooling2D, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Optimizer, Adam,RMSprop, SGD
from tensorflow.keras.regularizers import l2
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
#import argparse
import cv2
import os
import shutil
import math
import gc
from datetime import datetime

In [3]:
data_train = np.load('new_3000_data_train_224.npy')
labels_train = np.load('new_3000_labels_train_224.npy')

In [4]:
data_train.shape

(3000, 224, 224, 3)

In [5]:
#Shuffling Data
np.random.seed(12345)
indx=np.arange(data_train.shape[0])          
np.random.shuffle(indx)
data_train = data_train[indx]
labels_train = labels_train[indx]

In [6]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
  )

In [7]:
def create_model( opt ):
    lam = l2(1e-5)
    baseModel = InceptionV3(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    for layer in baseModel.layers:
        if hasattr(layer, 'kernel_regularizer'):
            setattr(layer, 'kernel_regularizer', lam)
    x = baseModel.output
    x = Flatten(name="flatten")(x)
    x = Dropout(0.1)(x)
    x = Dense(256, activation='relu', kernel_regularizer=lam)(x)
    x = Dropout(0.1)(x)
    x = Dense(3, activation="softmax", kernel_regularizer=lam)(x) #, kernel_regularizer=l2(0.0001)

    model = Model(inputs=baseModel.input, outputs=x)
    for layer in baseModel.layers: 
        layer.trainable =  True #False
    
    model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["acc"])
    
    return model

In [8]:
#model = create_model()
#model.summary()

In [9]:
labels = np.argmax(labels_train, axis=1)

In [10]:
from utils import plot_acc_loss
from utils import evaluation_metrics

In [11]:
from sklearn.model_selection import KFold, StratifiedKFold
skf = StratifiedKFold(n_splits =5, random_state = 7, shuffle = True)

In [12]:
!mkdir covid19_models_cross_val_InceptionV3_RMSprop
!mkdir covid19_models_cross_val_InceptionV3_Adam
!mkdir covid19_models_cross_val_InceptionV3_SGD

Un sous-répertoire ou un fichier covid19_models_cross_val_InceptionV3_RMSprop existe déjà.
Un sous-répertoire ou un fichier covid19_models_cross_val_InceptionV3_Adam existe déjà.
Un sous-répertoire ou un fichier covid19_models_cross_val_InceptionV3_SGD existe déjà.


In [ ]:
validation_accuracy = []
validation_loss = []

#base_model_opt = "InceptionV3_SGD"
#opt = SGD()

#base_model_opt = "InceptionV3_RMSprop"
#opt = RMSprop()

base_model_opt = "InceptionV3_Adam"
opt = Adam()

save_dir = 'covid19_models_cross_val_' + base_model_opt +'/' # A base directory to save results and models



fold_var = 1
for train_index, val_index in skf.split(np.zeros(len(data_train)),labels):
  
    training_data = data_train[train_index]
    training_labels = labels_train[train_index]
    validation_data = data_train[val_index]
    validation_labels = labels_train[val_index]

    model = create_model(opt)
        
    best_model_path = save_dir + 'model_' + base_model_opt + '_fold_' + str(fold_var) + '.h5'
    mcp_save = ModelCheckpoint(best_model_path, save_best_only=True, monitor='val_acc', mode='max', verbose=1)
    callbacks = [ mcp_save]
    print("[INFO] training K-Fold...step: " + str(fold_var))
    
    histories = []
    if opt == SGD():
        lrs = [ 0.001, 0.0001, 0.00001] # For SGD
    else:
        lrs = [ 0.0001, 0.00001, 0.000001] # For RMSprop and Adam
            
    lesEpochs = [40, 40, 20]
        
    BS_tr=32
    BS_val = 32
    for i in range(len(lrs)):
        opt.learning_rate = lrs[i]
        H = model.fit(
            trainAug.flow(training_data, training_labels, batch_size=BS_tr),
            #steps_per_epoch=len(training_data) // BS_tr ,
            validation_data=(validation_data, validation_labels),
            #validation_steps=len(validation_data) // BS_val ,
            epochs=lesEpochs[i], callbacks=callbacks, 
            shuffle=True)
        histories.append(H.history)
    
    #np.save(save_dir + 'history_' + base_model + '-' + str(fold_var) + '.npy', Hs.history)
    dict_history = {'loss': [], 'acc': [], 'val_loss': [], 'val_acc': []}
    for hist in histories:
        for key , value in dict_history.items():
            for a in hist[key]:
                dict_history[key].append(a )
    
    plot_acc_loss(dict_history, np.sum(lesEpochs))

    #Loading best model
    model.load_weights(best_model_path)
    #Saving weights
    model.save_weights(save_dir + 'model_' + base_model_opt + '_fold_' + str(fold_var) + '_weights.h5')
    results = model.evaluate(validation_data, validation_labels)
    results = dict(zip(model.metrics_names,results))
    
    validation_accuracy.append(results['acc'])
    validation_loss.append(results['loss'])
                       
    plt.figure()
    plt.show()

    evaluation_metrics(model, validation_data, validation_labels, fold_var, BS_val)
    
    #Saving accuracy and loss curves
    np.save(save_dir + base_model_opt + "_fold_" + str(fold_var) + "_acc.npy", dict_history["acc"])
    np.save(save_dir + base_model_opt + "_fold_" + str(fold_var) + "_loss.npy", dict_history["loss"])
    np.save(save_dir + base_model_opt + "_fold_" + str(fold_var) + "_val_acc.npy", dict_history["val_acc"])
    np.save(save_dir + base_model_opt + "_fold_" + str(fold_var) + "_val_loss.npy", dict_history["val_loss"])    

    tf.keras.backend.clear_session()
    
    fold_var = fold_var + 1 # Next fold

In [ ]:
print("Average accuracy : ", np.mean(validation_accuracy))
print("Average loss : ", np.mean(validation_loss))